<h1>Istrazivanje podataka u bioinformatici</h1>


<p> Predvidjanje neuredjenosti aminokiselina u sekvenci koriscenjem uslovnih slucajnih polja - CRF </p>

In [101]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import sklearn_crfsuite
from sklearn.metrics import make_scorer
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt

import numpy as np

import scipy.stats

import json

In [102]:
aa_facts = {
    "A": {
        "class": "Aliphatic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy": 1.8,
        "molecular_mass":89.094
    },
    "R":{
        "class": "Basic",
        "polarity": "Basic polar",
        "chargePH": "Positive",
        "hydropathy": -4.5,
        "molecular_mass":174.203
    },
    "N":{
        "class": "Amide",
        "polarity": "Polar",
        "chargePH": "Neutral",
        "hydropathy": -3.5,
        "molecular_mass":132.119
    },
    "D": {
        "class": "Acid",
        "polarity": "Acidic polar",
        "chargePH": "Negative",
        "hydropathy": -3.5,
        "molecular_mass":133.104
    },
    "C": {
        "class": "Sulfuric",
        "polarity": "Sulfuric",
        "chargePH": "Neutral",
        "hydropathy": 2.5,
        "molecular_mass":121.154
    },
    "Q": {
        "class": "Amide",
        "polarity": "Polar",
        "chargePH": "Neutral",
        "hydropathy": -3.5,
        "molecular_mass":146.146
    },
    "E": {
        "class": "Acid",
        "polarity": "Acidic polar",
        "chargePH": "Negative",
        "hydropathy": -3.5,
        "molecular_mass":147.131
    },
    "G": {
        "class": "Aliphatic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy": -0.4,
        "molecular_mass":75.067
    },
    "H": {
        "class": "Basic aromatic",
        "polarity": "Basic polar",
        "chargePH": "Positive, 10% Neutral, 90%",
        "hydropathy": -3.2,
        "molecular_mass":155.156
    },
    "I": {
        "class": "Aliphatic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy": 4.5,
        "molecular_mass":131.175
    },
    "L": {
        "class": "Aliphatic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy": 3.8,
        "molecular_mass":131.175
    },
    "K": {
        "class": "Basic",
        "polarity": "Basic polar",
        "chargePH": "Positive",
        "hydropathy": -3.9,
        "molecular_mass":146.189
    },
    "M": {
        "class": "Sulfuric",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy":1.9,
        "molecular_mass":149.208
    },
    "F": {
        "class": "Aromatic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy": 2.8,
        "molecular_mass":165.192
    },
    "P": {
        "class": "Cyclic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy": -1.6,
        "molecular_mass":115.132
    },
    "S":{
        "class": "Hydroxylic",
        "polarity": "Polar",
        "chargePH": "Neutral",
        "hydropathy": -0.8,
        "molecular_mass": 105.093
    },
    "T": {
        "class": "Hydroxylic",
        "polarity": "Polar",
        "chargePH": "Neutral",
        "hydropathy": -0.7,
        "molecular_mass": 119.119
    },
    "W": {
        "class": "Aromatic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy": -0.9,
        "molecular_mass": 204.228
    },
    "Y": {
        "class": "Aromatic",
        "polarity": "Polar",
        "chargePH": "Neutral",
        "hydropathy": -1.3,
        "molecular_mass": 181.191
    },
    "V": {
        "class": "Aliphatic",
        "polarity": "Nonpolar",
        "chargePH": "Neutral",
        "hydropathy":4.2,
        "molecular_mass": 117.148
    }
}

In [103]:
def aa_features(aa):
    global aa_facts
    named = aa_facts[aa]
    named['name'] = aa
    return named

<p>Računanje CRF-a. Trening i test podaci dobijaju se podelom podataka, sekvence sa osobinama i stanja neuredjenosti.</p>

In [104]:
def calculate_crf(sequences, order_state, radius = 3):
    featured_sequences = [seq_to_features(sequence, radius) for sequence in sequences]
    X_train, X_test, Y_train, Y_test = train_test_split(featured_sequences, order_state, test_size=0.33, random_state=42)
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1, 
        c2=0.1, 
        max_iterations=20,
        all_possible_transitions=False,
    )
    
    crf.fit(X_train, Y_train);

    labels = list(crf.classes_)
    print("Classes: ",labels)

    Y_pred = crf.predict(X_test)
    metrics.flat_f1_score(Y_test, Y_pred,
                          average='weighted', labels=labels)
    sorted_labels = sorted(
        labels,
        key=lambda name: (name[1:], name[0])
    )
    
    print(metrics.flat_classification_report(
        Y_test, Y_pred, labels=sorted_labels, digits=3))
    
    return Y_test, Y_pred

In [105]:
def read_data(datafile):
    with open(datafile, 'r') as j:
        json_data = json.load(j)
    return json_data

In [106]:
def seq_to_features(seq, rad):
    all_features = []
    for i in range(len(seq)):
        aa = aa_features(seq[i])
        features = {
            "class":aa["class"] ,
            "polarity": aa["polarity"],
            "chargePH": aa["chargePH"],
            "hydropathy": aa["hydropathy"],
            "molecular_mass": aa["molecular_mass"]
        }
        left = max(i - rad, 0)
        right = min(i + rad, len(seq))
        for j in range(left, i):
            aa = aa_features(seq[j])
            features.update({
                f'{j-i}:class':aa["class"] ,
                f"{j-i}:polarity": aa["polarity"],
                f"{j-i}:chargePH": aa["chargePH"],
                f"{j-i}:hydropathy": aa["hydropathy"],
                f"{j-i}:molecular_mass": aa["molecular_mass"]
            })
            #posmatranje okoline ispred tekuceg daje losije rezultate
#         for j in range(i+1, right):
#             aa = aa_features(seq[j])
#             features.update({
#                 f'+{j-i}:class':aa["class"] ,
#                 f"+{j-i}:polarity": aa["polarity"],
#                 f"+{j-i}:chargePH": aa["chargePH"],
#                 f"+{j-i}:hydropathy": aa["hydropathy"],
#                 f"+{j-i}:molecular_mass": aa["molecular_mass"]
#             })
        all_features.append(features)
    return all_features

In [107]:
def get_sequences(data):
    X = []
    y = []
    for entry in data:
        sequence = list(entry["sequence"])
        order_state = ['O' for x in sequence]
        for region in entry['regions']:
            if region['term_name'] == 'Disorder':
                start = region['start']
                end = region['end']
                order_state[start - 1:end] = ['D' for x in range(start - 1, end)] 
        #funkcija vraca sekvencu i raspored uredjenih/neuredjenih regiona
        X.append(sequence)
        y.append(order_state)

    return X, y

In [108]:
file = "disprot_data.json"
data = read_data(file)
data = data["data"]

sequences, order_state = get_sequences(data)

(Y_test, Y_pred) = calculate_crf(sequences, order_state, 4)

Classes:  ['O', 'D']
              precision    recall  f1-score   support

           D      0.487     0.074     0.128     20157
           O      0.823     0.982     0.895     88214

    accuracy                          0.813    108371
   macro avg      0.655     0.528     0.512    108371
weighted avg      0.760     0.813     0.753    108371



In [109]:
#sabiraju se neuredjeni regioni kako bi se znalo koliko neuredjenih ima u sekvenci
i = 0
for y in Y_pred:
    s = sum([1 if v == 'D' else 0 for v in y])
    if s > 0:
        print(i, s)
    i += 1

11 168
16 54
18 20
19 313
29 306
38 109
44 23
53 187
55 36
60 42
66 18
67 29
78 25
84 43
85 70
92 60
94 34
96 70
100 80
104 10
106 68
108 248
111 57
115 26
117 41
130 96
140 25
144 48
147 54
148 88
149 214
162 62
166 87
176 73
177 37
179 65
181 58
183 11


In [110]:
print(Y_test[108])
print(Y_pred[108])

['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D',